In [0]:
%fs ls /Volumes/my_catalog/default/mnt/sales_monthly/

In [0]:
%fs ls /Volumes/my_catalog/default/mnt/

In [0]:
%fs rm dbfs:/Volumes/my_catalog/default/mnt/schema/ -r

In [0]:
%sql
select * from my_catalog.default.sales_monthly
-- drop table my_catalog.default.sales_monthly

In [0]:
%sql
desc history my_catalog.default.sales_monthly

In [0]:
from pyspark.sql.functions import *

df = spark.readStream \
    .format("cloudFiles") \
    .option("cloudFiles.format", "csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .option("cloudFiles.schemaLocation", "/Volumes/my_catalog/default/mnt/schema/monthly-sales/")\
    .load("/Volumes/my_catalog/default/mnt/sales_monthly/")\
    .withColumn('file_name',col("_metadata.file_path"))

df.writeStream \
    .format("delta") \
    .option("checkpointLocation", "/Volumes/my_catalog/default/mnt/checkpoints/sales") \
    .outputMode("append") \
    .option('mergeSchema','true')\
    .trigger(availableNow=True) \
    .toTable("my_catalog.default.sales_monthly")  # Creates Delta table on the fly


In [0]:
df = spark.readStream.format("cloudFiles") \
    .option("cloudFiles.format", "csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .option("cloudFiles.schemaLocation", "/Volumes/my_catalog/default/mnt/schema/monthly-sales/") \
    .load("/Volumes/my_catalog/default/mnt/sales_monthly/")



In [0]:
df.writeStream \
    .format("console") \
    .option("checkpointLocation", "/Volumes/my_catalog/default/mnt/checkpoints/sales")\
    .outputMode("append") \
    .trigger(once=True)\
    .start()
